# Week 4-5: Motors and Actuators

## Topics Covered:
- DC Motors (Brushed and Brushless)
- Servo Motors and PWM Control
- Stepper Motors and Drivers
- Linear Motors
- H-Bridge Motor Controllers
- Motor Speed Control
- Encoder Feedback

## LAB: Robotic Arm Project

---

## Setup

In [ ]:
!pip install matplotlib numpy scipy schemdraw -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import schemdraw
import schemdraw.elements as elm
from scipy import signal

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. DC Motors

**DC Motor**: Converts electrical energy to mechanical rotation

### Types:
- **Brushed DC**: Simple, inexpensive, requires maintenance
- **Brushless DC (BLDC)**: More efficient, longer life, requires controller

### Key Parameters:
- Voltage rating
- Current draw (no-load and stall)
- RPM (revolutions per minute)
- Torque
- Efficiency

In [ ]:
# DC Motor speed vs voltage simulation
voltage = np.linspace(0, 12, 100)
rpm_no_load = voltage * 1000  # ~1000 RPM per volt (typical)

# With load (torque)
torque_percent = np.array([0, 25, 50, 75, 100])

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(voltage, rpm_no_load, 'b-', linewidth=2, label='No Load')
for torque in torque_percent:
    rpm_loaded = rpm_no_load * (1 - torque/150)
    if torque > 0:
        plt.plot(voltage, rpm_loaded, linewidth=2, label=f'{torque}% Torque')

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Speed (RPM)', fontsize=12)
plt.title('DC Motor Speed vs Voltage', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Current vs torque
plt.subplot(1, 2, 2)
torque_values = np.linspace(0, 1, 100)  # Normalized
current_no_load = 0.1  # 100mA no-load
current_stall = 2.0  # 2A stall
current = current_no_load + (current_stall - current_no_load) * torque_values

plt.plot(torque_values * 100, current, 'r-', linewidth=2)
plt.xlabel('Torque (% of max)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Motor Current vs Torque', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Servo Motors

**Servo Motor**: DC motor with position feedback and control electronics

### Characteristics:
- Position control (typically 0-180°)
- PWM signal input (1-2ms pulse, 20ms period)
- Built-in gearbox
- Internal feedback potentiometer

### Control Signal:
- 1.0ms pulse = 0°
- 1.5ms pulse = 90°
- 2.0ms pulse = 180°

In [ ]:
# Servo PWM signal simulation
time = np.linspace(0, 0.06, 1000)  # 60ms (3 periods)
period = 0.02  # 20ms

def generate_servo_pwm(angle, time, period=0.02):
    pulse_width = 0.001 + (angle / 180) * 0.001  # 1ms to 2ms
    signal = np.zeros_like(time)
    for t_start in np.arange(0, time[-1], period):
        mask = (time >= t_start) & (time < t_start + pulse_width)
        signal[mask] = 5
    return signal

angles = [0, 90, 180]
colors = ['r', 'g', 'b']

fig, axes = plt.subplots(len(angles), 1, figsize=(12, 10))

for idx, (angle, color) in enumerate(zip(angles, colors)):
    pwm_signal = generate_servo_pwm(angle, time)
    axes[idx].plot(time * 1000, pwm_signal, color=color, linewidth=2)
    axes[idx].fill_between(time * 1000, 0, pwm_signal, alpha=0.3, color=color)
    axes[idx].set_ylabel('Voltage (V)', fontsize=11)
    axes[idx].set_title(f'Servo PWM Signal for {angle}°', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim(-0.5, 6)

axes[-1].set_xlabel('Time (ms)', fontsize=12)
plt.tight_layout()
plt.show()

## 3. Stepper Motors

**Stepper Motor**: Motor that moves in discrete steps

### Types:
- **Unipolar**: 5 or 6 wires, easier to drive
- **Bipolar**: 4 wires, more torque

### Key Features:
- Precise position control
- No feedback required (open-loop)
- Steps per revolution: 200 (1.8°), 400 (0.9°)
- Full step, half step, microstepping modes

### Applications:
- 3D printers
- CNC machines
- Robotics
- Camera gimbals

In [ ]:
# Stepper motor stepping sequence
steps = 200  # Steps per revolution
angle_per_step = 360 / steps

# Full step sequence (4 phases)
step_sequence = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Simulate rotation
num_steps = 20
time_per_step = 0.01  # 10ms
time_total = num_steps * time_per_step
time_array = np.arange(0, time_total, time_per_step)

angles = np.arange(0, num_steps) * angle_per_step

# Phase signals
phase_signals = np.zeros((4, num_steps))
for i in range(num_steps):
    phase_signals[:, i] = step_sequence[i % 4]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Position over time
ax1.step(time_array, angles, 'b-', linewidth=2, where='post')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Angle (degrees)', fontsize=12)
ax1.set_title('Stepper Motor Position', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Phase activation
for phase in range(4):
    ax2.step(time_array, phase_signals[phase] + phase * 1.5, 
             linewidth=2, where='post', label=f'Phase {phase+1}')

ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Phase State', fontsize=12)
ax2.set_title('Stepper Motor Phase Sequence (Full Step)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## LAB 2: Robotic Arm Project

### Objective:
Design and simulate a 3-DOF (Degrees of Freedom) robotic arm using servo motors.

### Components:
- 3x Servo Motors (base, shoulder, elbow)
- Microcontroller (Arduino/ESP32)
- Power supply (5-6V, 2A+)
- Mechanical structure

### Design Considerations:
1. Calculate reach and workspace
2. Determine servo torque requirements
3. Plan movement sequences
4. Implement inverse kinematics

In [ ]:
# Robotic arm kinematics simulation
def forward_kinematics(theta1, theta2, theta3, L1=10, L2=8, L3=6):
    '''Calculate end effector position from joint angles'''
    theta1_rad = np.radians(theta1)
    theta2_rad = np.radians(theta2)
    theta3_rad = np.radians(theta3)
    
    # Joint positions
    x0, y0 = 0, 0  # Base
    x1 = L1 * np.cos(theta1_rad)
    y1 = L1 * np.sin(theta1_rad)
    
    x2 = x1 + L2 * np.cos(theta1_rad + theta2_rad)
    y2 = y1 + L2 * np.sin(theta1_rad + theta2_rad)
    
    x3 = x2 + L3 * np.cos(theta1_rad + theta2_rad + theta3_rad)
    y3 = y2 + L3 * np.sin(theta1_rad + theta2_rad + theta3_rad)
    
    return [(x0, y0), (x1, y1), (x2, y2), (x3, y3)]

# Test different poses
poses = [
    (0, 45, 45, 'Extended'),
    (45, -30, 30, 'Reaching'),
    (90, 0, 0, 'Vertical'),
    (-45, 60, -30, 'Folded')
]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (theta1, theta2, theta3, name) in enumerate(poses):
    points = forward_kinematics(theta1, theta2, theta3)
    
    # Extract x and y coordinates
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    
    # Plot arm
    axes[idx].plot(x_coords, y_coords, 'b-o', linewidth=3, markersize=10, label='Arm')
    axes[idx].plot(x_coords[-1], y_coords[-1], 'r*', markersize=20, label='End Effector')
    
    # Draw workspace circle
    theta_circle = np.linspace(0, 2*np.pi, 100)
    max_reach = 10 + 8 + 6
    circle_x = max_reach * np.cos(theta_circle)
    circle_y = max_reach * np.sin(theta_circle)
    axes[idx].plot(circle_x, circle_y, 'g--', alpha=0.3, label='Max Reach')
    
    axes[idx].set_xlim(-25, 25)
    axes[idx].set_ylim(-5, 25)
    axes[idx].set_aspect('equal')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlabel('X (cm)', fontsize=11)
    axes[idx].set_ylabel('Y (cm)', fontsize=11)
    axes[idx].set_title(f'{name}\nθ1={theta1}° θ2={theta2}° θ3={theta3}°', 
                        fontsize=12, fontweight='bold')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print('Robotic Arm Lab 2: Design your own arm!')
print('1. Choose servo motors based on torque requirements')
print('2. Build mechanical structure')
print('3. Implement control software')
print('4. Test and calibrate movements')